# Рубежный контроль №2
Вариант **№4**

Студент: **Кучеренко М.А.**  
Группа: **ИУ5-21М**

### Классификатор №1
LogisticRegression

### Классификатор №2
Multinomial Naive Bayes (MNB)

Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.
Необходимо сформировать два варианта векторизации признаков - на основе `CountVectorizer` и на основе `TfidfVectorizer`.
Для каждого метода необходимо оценить качество классификации. Сделайте вывод о том, какой вариант векторизации признаков в паре с каким классификатором показал лучшее качество.